# Submission


In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from pathlib import Path
from tqdm import tqdm
import os
import gzip
import subprocess
from sklearn.model_selection import train_test_split

import tensorflow as tf

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import regularizers

import matplotlib.pyplot as plt
%matplotlib inline

## Preprocessing for training

In [2]:
data_path = 'testing/'

df_train = pd.read_csv(data_path + 'Train.csv')

def load_and_resize_image(filepath, shape):
    """
    Loads and resizes an image from a filepath.
    """
    raw_img = tf.io.read_file(filepath)
    img_tensor = tf.io.decode_png(raw_img, channels=3)
    img_tensor = tf.image.resize(img_tensor, tf.constant(shape[:2]))
    return tf.cast(img_tensor, tf.float32) / 255.

# load data
imgs = np.zeros((len(df_train), 50, 50, 3))
for i in tqdm(range(len(df_train)), colour='green'):
    imgs[i] = load_and_resize_image(data_path + df_train['Path'].iloc[i], (50, 50, 3))

labels = df_train['ClassId'].to_numpy()
train_data, test_data, train_labels, test_labels = train_test_split(imgs, labels, test_size=0.2)
train_labels = train_labels.reshape(-1, 1)
test_labels = test_labels.reshape(-1, 1)

# turn RGB to grayscale
scaled_test_img = np.expand_dims(np.dot(test_data, [0.2990, 0.5870, 0.1140]), axis=3)
scaled_train_img = np.expand_dims(np.dot(train_data,[0.2990,0.5870,0.1140]),axis=3)

# normalizing
def normalize(imgs):
  mean = np.mean(imgs, axis=0)
  std = np.std(imgs, axis=0)
  imgs = (imgs-mean) / std
  return imgs

scaled_test_img = normalize(scaled_test_img)
scaled_train_img = normalize(scaled_train_img)

100%|██████████| 39209/39209 [02:25<00:00, 269.72it/s]


# CNN

In [8]:
earlystopping = EarlyStopping(monitor='val_loss',
                              patience=10,
                              min_delta=0.001,
                              verbose=1)

checkpoint_CNN = ModelCheckpoint(filepath='models/checkpoint',
                                 save_freq='epoch',
                                 monitor='val_loss',
                                 save_best_only=True,
                                 save_weights_only=True,
                                 verbose=1)
def CNN():
    model = Sequential([
        tf.keras.Input(shape=(50, 50, 1)),
        Conv2D(32, kernel_size=3, activation='relu', padding='SAME'),
        BatchNormalization(),
        Conv2D(32, kernel_size=3, activation='relu', padding='SAME'),
        BatchNormalization(),
        MaxPooling2D(2),
        Dropout(0.3),
        Flatten(),
        Dense(32, activation='relu',kernel_regularizer=regularizers.l2(1e-5)),
        Dropout(0.3),
        Dense(43, activation='softmax')])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    return model

## Starting training

Uncomment the following to start training.

In [ ]:
# model = CNN()
# history = model.fit(scaled_train_img, 
#                     train_labels, 
#                     epochs=30,
#                     validation_split=0.2,
#                     verbose=1,
#                     callbacks=[earlystopping,checkpoint_CNN])

## Testing

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss vs. epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')
plt.show()

In [ ]:
plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.title('Accuracy vs. epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')
plt.show()

In [11]:
model = CNN()
model.summary()
model.load_weights('models/checkpoint')

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 50, 50, 32)        320       
                                                                 
 batch_normalization_12 (Bat  (None, 50, 50, 32)       128       
 chNormalization)                                                
                                                                 
 conv2d_13 (Conv2D)          (None, 50, 50, 32)        9248      
                                                                 
 batch_normalization_13 (Bat  (None, 50, 50, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 25, 25, 32)       0         
 2D)                                                             
                                                      

In [12]:
test_loss, test_accuracy = model.evaluate(scaled_test_img, test_labels, verbose=0)
print(f"Test loss: {test_loss}")
print(f"Test accuracy: {test_accuracy}")

Test loss: 0.07643140852451324
Test accuracy: 0.9931139945983887


## Submission

In [8]:
def submission(filepath, name_csv):
    """"
        Returns preprocessed data.  
        
        Args:  
            - filepath: str, path to the data folder
            - name_csv: str, name of the csv file
            
        Returns:  
            - test_acc: float, accuracy of the model on the test set
        """
    def preprocessing(data_path, name_csv): 
        df_train = pd.read_csv(data_path + name_csv)

        def load_and_resize_image(filepath, shape):
            """
            Loads and resizes an image from a filepath.
            """
            raw_img = tf.io.read_file(filepath)
            img_tensor = tf.io.decode_png(raw_img, channels=3)
            img_tensor = tf.image.resize(img_tensor, tf.constant(shape[:2]))
            return tf.cast(img_tensor, tf.float32) / 255.

        # load data
        imgs = np.zeros((len(df_train), 50, 50, 3))
        
        for i in tqdm(range(len(df_train)), colour='green', ascii=" >="):
            imgs[i] = load_and_resize_image(data_path + df_train['Path'].iloc[i], (50, 50, 3))

        labels = df_train['ClassId'].to_numpy()

        test_labels = labels.reshape(-1, 1)
        
        # turn RGB to grayscale
        scaled_test_img = np.expand_dims(np.dot(imgs, [0.2990, 0.5870, 0.1140]), axis=3)

        # normalizing
        def normalize(imgs):
            mean = np.mean(imgs, axis=0)
            std = np.std(imgs, axis=0)
            imgs = (imgs-mean) / std
            return imgs

        scaled_test_img = normalize(scaled_test_img)
        
        return scaled_test_img, test_labels
    def CNN():
        model = Sequential([
            tf.keras.Input(shape=(50, 50, 1)),
            Conv2D(32, kernel_size=3, activation='relu', padding='SAME'),
            BatchNormalization(),
            Conv2D(32, kernel_size=3, activation='relu', padding='SAME'),
            BatchNormalization(),
            MaxPooling2D(2),
            Dropout(0.3),
            Flatten(),
            Dense(32, activation='relu',kernel_regularizer=regularizers.l2(1e-5)),
            Dropout(0.3),
            Dense(43, activation='softmax')])
        
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
        return model
    scaled_test_img, test_labels = preprocessing(filepath, name_csv)
    model = CNN()
    model.load_weights('models/checkpoint')
    test_loss, test_accuracy = model.evaluate(scaled_test_img, test_labels, verbose=0)
    print(f"Test loss: {test_loss}")
    print(f"Test accuracy: {test_accuracy}")
    return test_accuracy

In [9]:
submission('testing/', 'Train.csv')

100%|==========| 39209/39209 [02:33<00:00, 255.14it/s]


Test loss: 0.08162304013967514
Test accuracy: 0.9921446442604065


0.9921446442604065